## BTCNet

Welcome to BTCNet, a Bitcoin trading bot using machine learning.

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

We create a long short term memory recurrent neural network to predict the price of Bitcoin 1 hour in the future. We train the network on historical hourly pricing data along with volume.

Thesis: The price of Bitcoin can be sufficiently predicted by momentum and volume.

FUD, FOMO, Whales, and manipulation lead to large price swings driven by momentum in short periods of time.

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


Import the historical price data.

We create our own Dataset class for our dataset.

In [44]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader

class BTCDataset(Dataset):
    def __init__(self, csv):
        self.btcFrame = pd.read_csv(csv)
        self.targets = self.btcFrame.iloc[:, 4:5]
        
    def __len__(self):
        return len(self.btcFrame)
    
    def __getitem__(self, i):
        item = self.btcFrame.iloc[i, 1:2].values
        item = item.astype("float")
        return item
    
    def getTarget(self, start, batch_size):
        target = self.targets.iloc[i:i+batch_size, :].values
        target = torch.from_numpy(target)
        target = target.float()
        target = target.unsqueeze(0)
        return target
    
    def input_size(self):
        return len(self[0])

Now we import our dataset from a csv file.

In [45]:
btcData = BTCDataset("bitcoinPrices.csv")

for i in range(10):
    sample = btcData[i]
    print(sample, f"Target is {btcData.getTarget(i, batch_size)}")

[300.] Target is tensor([[[300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.]]])
[300.] Target is tensor([[[300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.]]])
[300.] Target is tensor([[[300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.]]])
[300.] Target is tensor([[[300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.]]])
[300.] Target is tensor([[[300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.]]])
[300.] Target is tensor([[[300.],
         [300.],
    

In [46]:
batch_size = 10

dataloader = DataLoader(btcData, batch_size=batch_size, shuffle=False, num_workers=2)

for i, sample in enumerate(dataloader):
    print(i, sample)
    
    if i == 100:
        break

0 tensor([[300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.]], dtype=torch.float64)
1 tensor([[300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.]], dtype=torch.float64)
2 tensor([[300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.]], dtype=torch.float64)
3 tensor([[300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.]], dtype=torch.float64)
4 tensor([[300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.],
        [300.]], dtype=torch.float64)
5 tensor([[300.],
        [370.],
        [370.],
        [370.],
        [370.]

70 tensor([[370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.]], dtype=torch.float64)
71 tensor([[370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.]], dtype=torch.float64)
72 tensor([[370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.]], dtype=torch.float64)
73 tensor([[370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.]], dtype=torch.float64)
74 tensor([[370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.],
        [370.]], dtype=torch.float64)
75 tensor([[370.],
        [370.],
        [370.],
        [370.],
        

In [47]:
class BTCNet(nn.Module):
    def __init__(self, input_size, batch_size, hidden_size, output_size):
        super(BTCNet, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size)
        self.fc1 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        
        self.hidden = self.init_hidden()
       
    def forward(self, data):
        lstm_out, self.hidden = self.lstm(data.float(), self.hidden)
        lin_out = self.fc1(lstm_out)
        out = self.relu(lin_out)
        out = out.float()
        return out
    
    def init_hidden(self):
        return (torch.zeros(1, self.batch_size, self.hidden_size), torch.zeros(1, self.batch_size, self.hidden_size))

In [50]:
btcModel = BTCNet(input_size=btcData.input_size(), batch_size=batch_size, hidden_size=5, output_size=1)

data_iter = iter(dataloader)

learning_rate = 0.0001

mseLoss = nn.MSELoss()
optimizer = optim.SGD(btcModel.parameters(), lr=learning_rate)

## Training

First, we will see what the predicted prices are before training.

In [51]:
with torch.no_grad():
    initial_in = next(data_iter)
    initial_in = initial_in.unsqueeze(0)
    initial_out = btcModel(initial_in)
    print("Open price:", initial_in)
    print("Close price:", initial_out)

Open price: tensor([[[300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.],
         [300.]]], dtype=torch.float64)
Close price: tensor([[[0.4737],
         [0.4737],
         [0.4737],
         [0.4737],
         [0.4737],
         [0.4737],
         [0.4737],
         [0.4737],
         [0.4737],
         [0.4737]]])


In [53]:
num_epochs = 10

for i in range(num_epochs):
    for item in data_iter:
        btcModel.zero_grad()
        btcModel.hidden = btcModel.init_hidden()

        item = item.unsqueeze(0)
        next_out = btcModel(item)

        next_target = btcData.getTarget(i+1, batch_size)

        loss = mseLoss(next_out, next_target)
        print("Epoch: ", i, " Loss: ", loss)
        loss.backward()
        optimizer.step()

Epoch:  0  Loss:  tensor(89150.6797, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(89094.3438, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(89038.0391, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(88981.7891, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(88925.5625, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(88869.3594, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(88813.2031, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(88757.0781, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(88700.9922, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(88644.9375, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(88588.9219, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(88532.9453, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(88477., grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(88421.0938, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(88365.2188, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(88309.3828, grad_fn

Epoch:  0  Loss:  tensor(81342.7734, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(81291.3750, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(81240.0156, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(81188.6641, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(81137.3672, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(81086.0938, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(81034.8594, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(80983.6484, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(80932.4688, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(80881.3281, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(80830.2344, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(80779.1562, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(80728.1094, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(80677.0781, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(80626.1016, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(80575.1562, gra

Epoch:  0  Loss:  tensor(73751.0312, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(73704.4297, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(73657.8594, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(73611.3203, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(73564.7891, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(73518.3125, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(73471.8516, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(73425.4219, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(73379.0391, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(73332.6641, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(73286.3281, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(73240.0078, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(73193.7266, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(73147.4844, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(73101.2578, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(73055.0547, gra

Epoch:  0  Loss:  tensor(67462.1953, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(67419.5781, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(67376.9688, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(67334.3906, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(67291.8438, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(67249.3203, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(67206.8281, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(67164.3672, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(67121.9141, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(67079.5000, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(67037.1094, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(66994.7500, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(66952.4141, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(66910.1094, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(66867.8203, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(66825.5781, gra

Epoch:  0  Loss:  tensor(61553.7930, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(61514.8984, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(61476.0234, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(61437.1758, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(61398.3594, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(61359.5586, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(61320.7852, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(61282.0391, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(61243.3125, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(61204.6133, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(61165.9336, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(61127.2812, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(61088.6562, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(61050.0508, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(61011.4727, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(60972.9219, gra

...

Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn

Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn

Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn

Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn=<MseLossBackward>)
Epoch:  0  Loss:  tensor(0.0015, grad_fn

KeyboardInterrupt: 